In [2]:
from multiprocessing.spawn import _main
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import sampler

import torchvision.transforms as T

In [ ]:
X_train_means = pd.read_csv('/home/ubuntu/CS231N/cs231n_final_project/data_preprocessing/X_train_channel_means.csv',index_col=False).to_numpy()
X_train_means[:,1]
# norm_trans = T.Normalize()

In [ ]:
class FCNet(nn.Module):
    def __init__(self, input_size, num_classes):
        super(FCNet, self).__init__()
        self.input_size = input_size
        self.fc1 = nn.Linear(self.input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, self.input_size)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

def load_data(train =True,valid = True,test = False):
    # Load the raw CIFAR-10 data.
    data_dir = '/home/ubuntu/CS231N/data/split_datasets/'
    # data_dir = "../../data/"
    X_train,y_train,X_valid,y_valid,X_test,y_test = None, None, None, None, None, None
    if train:
        X_train = pd.read_pickle(data_dir + "train_data.pkl").to_numpy()
        y_train = pd.read_pickle(data_dir + "train_labels.pkl").to_numpy()
        y_train = y_train.flatten().astype(np.int64)
        print('Training data shape: ', X_train.shape)
        print('Training labels shape: ', y_train.shape)
    if valid:
        X_valid = pd.read_pickle(data_dir + "valid_data.pkl").to_numpy()
        y_valid = pd.read_pickle(data_dir + "valid_labels.pkl").to_numpy()
        y_valid = y_valid.flatten().astype(np.int64)
        print('Validation data shape: ', X_valid.shape)
        print('Validation labels shape: ', y_valid.shape)
    if test:
        X_test = pd.read_pickle(data_dir + "test_data.pkl").to_numpy()
        y_test = pd.read_pickle(data_dir + "test_labels.pkl").to_numpy()
        y_test = y_test.flatten().astype(np.int64)
        print('Test data shape: ', X_test.shape)
        print('Test labels shape: ', y_test.shape)
    # As a sanity check, we print out the size of the data we output.
    # if didn;t load the data, it is None
    return X_train,y_train,X_valid,y_valid,X_test,y_test

In [ ]:
def train(model, optimizer, epochs=1):    
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

        num_correct, num_samples = check_accuracy(loader_val, model)
        epoch_acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f) after epoch %d' % (num_correct, num_samples, 100 * epoch_acc, e+1))

    return epoch_acc